In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir "/content/drive/Shareddrives/VoxCeleb1/Dev_Filtered/wav/id10001"
!mkdir "/content/drive/Shareddrives/VoxCeleb1/Dev_Filtered/wav/id10001/7gWzIy6yIIk"
!deepFilter -i /content/drive/Shareddrives/VoxCeleb1/Dev_Augmented/wav/id10001/7gWzIy6yIIk -o "/content/drive/Shareddrives/VoxCeleb1/Dev_Filtered/wav/id10001/7gWzIy6yIIk"
# !deepFilter --help

/usr/local/lib/python3.11/dist-packages/df/io.py:9: UserWarning: `torchaudio.backend.common.AudioMetaData` has been moved to `torchaudio.AudioMetaData`. Please update the import path.
  from torchaudio.backend.common import AudioMetaData
2025-03-23 20:45:45 | INFO     | DF | Running on torch 2.6.0+cu124
2025-03-23 20:45:45 | INFO     | DF | Running on host 9673c71c2e91
fatal: not a git repository (or any of the parent directories): .git
2025-03-23 20:45:45 | INFO     | DF | Loading model settings of DeepFilterNet3
2025-03-23 20:45:45 | INFO     | DF | Using DeepFilterNet3 model at /root/.cache/DeepFilterNet/DeepFilterNet3
2025-03-23 20:45:45 | INFO     | DF | Initializing model `deepfilternet3`
2025-03-23 20:45:46 | INFO     | DF | Found checkpoint /root/.cache/DeepFilterNet/DeepFilterNet3/checkpoints/model_120.ckpt.best with epoch 120
2025-03-23 20:45:46 | INFO     | DF | Running on device cuda:0
2025-03-23 20:45:46 | INFO     | DF | Model loaded
2025-03-23 20:45:46 | WARNING  | DF | 

In [ ]:
original, sr1 = torchaudio.load("/content/drive/Shareddrives/VoxCeleb1/Dev/wav/id10001/7gWzIy6yIIk/00001.wav")
noisy, sr = torchaudio.load("/content/drive/Shareddrives/VoxCeleb1/Dev_Augmented/wav/id10001/7gWzIy6yIIk/00001.wav")
filtered, sr2 = torchaudio.load("/content/drive/Shareddrives/VoxCeleb1/Dev_Filtered/wav/id10001/7gWzIy6yIIk/00001_DeepFilterNet3.wav")
display(Audio(original, rate = sr1))
display(Audio(noisy, rate = sr))
display(Audio(filtered, rate=sr2))

# install required packages

In [4]:
!pip install torch torchaudio
!pip install DeepFilterNet
!pip install pystoi pesq tqdm pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 115.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 98.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjit

In [ ]:
!pip install torch torchaudio
!pip install DeepFilterNet
!pip install pystoi pesq tqdm pandas

# Write script to .py file

In [1]:
%%writefile process_voxceleb.py
import os
import subprocess
from tqdm import tqdm
import argparse
import glob

def ensure_dir(directory):
    """Create directory if it doesn't exist"""
    if not os.path.exists(directory):
        os.makedirs(directory)
        return True
    return False

def process_voxceleb(source_dir, target_dir, dry_run=False):
    """
    Process VoxCeleb dataset with DeepFilterNet

    Args:
        source_dir: Path to source VoxCeleb directory (e.g., /content/drive/Shareddrives/VoxCeleb1/Dev_Augmented/wav)
        target_dir: Path to target directory for filtered files (e.g., /content/drive/Shareddrives/VoxCeleb1/Dev_Filtered/wav)
        dry_run: If True, only print commands without executing
    """
    # Get all speaker IDs (directories starting with 'id')
    speaker_dirs = sorted([d for d in os.listdir(source_dir) if d.startswith('id')])

    # Create a log file to track progress
    log_file = os.path.join(os.path.dirname(target_dir), "deepfilter_processing.log")

    # Counter for statistics
    stats = {
        'total_speakers': len(speaker_dirs),
        'total_sessions': 0,
        'completed_sessions': 0,
        'failed_sessions': 0
    }

    print(f"Found {stats['total_speakers']} speaker directories")

    # Loop through all speaker IDs
    for speaker_id in tqdm(speaker_dirs, desc="Processing speakers"):
        # Create the speaker directory in target
        speaker_target_dir = os.path.join(target_dir, speaker_id)
        ensure_dir(speaker_target_dir)

        # Get session directories for this speaker
        speaker_source_dir = os.path.join(source_dir, speaker_id)
        session_dirs = sorted(os.listdir(speaker_source_dir))
        stats['total_sessions'] += len(session_dirs)

        # Loop through all sessions for this speaker
        for session_id in session_dirs:
            session_source_dir = os.path.join(speaker_source_dir, session_id)
            session_target_dir = os.path.join(speaker_target_dir, session_id)

            # Create the session directory in target
            is_new_dir = ensure_dir(session_target_dir)

            # Construct the deepFilter command
            cmd = [
                'deepFilter',
                '-i', session_source_dir,
                '-o', session_target_dir
            ]

            # Only process if the directory was newly created or is empty
            if is_new_dir or len(os.listdir(session_target_dir)) == 0:
                cmd_str = ' '.join(cmd)
                print(f"Running: {cmd_str}")

                if not dry_run:
                    try:
                        # Execute the deepFilter command
                        result = subprocess.run(cmd, capture_output=True, text=True)

                        # Check if command was successful
                        if result.returncode == 0:
                            stats['completed_sessions'] += 1
                            # Log success
                            with open(log_file, 'a') as f:
                                f.write(f"SUCCESS: {session_source_dir} -> {session_target_dir}\n")
                        else:
                            stats['failed_sessions'] += 1
                            # Log error
                            with open(log_file, 'a') as f:
                                f.write(f"ERROR: {session_source_dir} -> {session_target_dir}\n")
                                f.write(f"  {result.stderr}\n")
                            print(f"Error processing {session_source_dir}: {result.stderr}")
                    except Exception as e:
                        stats['failed_sessions'] += 1
                        # Log exception
                        with open(log_file, 'a') as f:
                            f.write(f"EXCEPTION: {session_source_dir} -> {session_target_dir}\n")
                            f.write(f"  {str(e)}\n")
                        print(f"Exception processing {session_source_dir}: {str(e)}")
            else:
                print(f"Skipping {session_target_dir} (already processed)")
                stats['completed_sessions'] += 1

    # Print summary statistics
    print("\nProcessing Summary:")
    print(f"Total speakers: {stats['total_speakers']}")
    print(f"Total sessions: {stats['total_sessions']}")
    print(f"Completed sessions: {stats['completed_sessions']}")
    print(f"Failed sessions: {stats['failed_sessions']}")
    print(f"Log file: {log_file}")

def main():
    parser = argparse.ArgumentParser(description="Process VoxCeleb dataset with DeepFilterNet")
    parser.add_argument("--source", type=str, required=True,
                       help="Source directory (e.g., /content/drive/Shareddrives/VoxCeleb1/Dev_Augmented/wav)")
    parser.add_argument("--target", type=str, required=True,
                       help="Target directory (e.g., /content/drive/Shareddrives/VoxCeleb1/Dev_Filtered/wav)")
    parser.add_argument("--dry-run", action="store_true",
                       help="Print commands without executing")
    parser.add_argument("--resume", action="store_true",
                       help="Resume processing (skip already processed directories)")

    args = parser.parse_args()

    # Ensure target directory exists
    ensure_dir(args.target)

    # Process the dataset
    process_voxceleb(args.source, args.target, args.dry_run)

if __name__ == "__main__":
    main()

Writing process_voxceleb.py


In [5]:
source_dir = "/content/drive/Shareddrives/VoxCeleb1/Test_Augmented/wav/wav"
target_dir = "/content/drive/Shareddrives/VoxCeleb1/Test_Filtered/wav"

!python process_voxceleb.py --source "$source_dir" --target "$target_dir"

Found 31 speaker directories
Processing speakers:   0% 0/31 [00:00<?, ?it/s]Running: deepFilter -i /content/drive/Shareddrives/VoxCeleb1/Test_Augmented/wav/wav/id10270/5r0dWxy17C8 -o /content/drive/Shareddrives/VoxCeleb1/Test_Filtered/wav/id10270/5r0dWxy17C8
Running: deepFilter -i /content/drive/Shareddrives/VoxCeleb1/Test_Augmented/wav/wav/id10270/5sJomL_D0_g -o /content/drive/Shareddrives/VoxCeleb1/Test_Filtered/wav/id10270/5sJomL_D0_g
Running: deepFilter -i /content/drive/Shareddrives/VoxCeleb1/Test_Augmented/wav/wav/id10270/8jEAjG6SegY -o /content/drive/Shareddrives/VoxCeleb1/Test_Filtered/wav/id10270/8jEAjG6SegY
Running: deepFilter -i /content/drive/Shareddrives/VoxCeleb1/Test_Augmented/wav/wav/id10270/GWXujl-xAVM -o /content/drive/Shareddrives/VoxCeleb1/Test_Filtered/wav/id10270/GWXujl-xAVM
Running: deepFilter -i /content/drive/Shareddrives/VoxCeleb1/Test_Augmented/wav/wav/id10270/OXdd7Gmluts -o /content/drive/Shareddrives/VoxCeleb1/Test_Filtered/wav/id10270/OXdd7Gmluts
Running: 